In [ ]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(1)
while True:
    # 读取图像
    ret,frame = cap.read()
    gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY) # 灰度图
    _,thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU) # 图像阈值
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(5,5)) # 卷积核
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel) # 图像开运算，去除噪点
    contours,hierarchy = cv.findContours(thresh,3,2) # 查找轮廓
    img = np.copy(frame)
    for cnt in contours:
        area = cv.contourArea(cnt) # 面积
        perimeter = cv.arcLength(cnt,True) # 周长
        # 面积筛选
        if area > 4000 and area < 20000:
            cv.drawContours(img,[cnt],0,(0,0,255),5) # 画出轮廓
            # 获得质心
            M = cv.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            # 多边形逼近，获得角点判断形状
            approx = cv.approxPolyDP(cnt,0.02*perimeter,True)
            if len(approx) == 4:
                text = 'rectangle'
            elif len(approx) == 5:
                text = 'pentagon'
            elif len(approx) == 3:
                text = 'triangle'
            elif len(approx) == 8:
                text = 'circle'
            else:
                text = 'None'
            cv.putText(img,text,(cx,cy),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv.LINE_AA)
    cv.imshow('image',img)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.destroyAllWindows()